In [1]:
pip install pillow numpy sklearn matplotlib


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

Code for Paint-by-number kit

In [2]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [3]:
# Load the image
def load_image(image_path):
    return Image.open(image_path)

In [4]:
# Reduce the number of colors using KMeans clustering
def reduce_colors(image, num_colors=10):
    # Convert the image to RGB and then to a NumPy array
    img_rgb = image.convert("RGB")
    img_array = np.array(img_rgb)
    
    # Reshape the image to a 2D array of pixels
    pixels = img_array.reshape(-1, 3)
    
    # Apply KMeans to cluster the pixels into 'num_colors' clusters
    kmeans = KMeans(n_clusters=num_colors, random_state=0).fit(pixels)
    
    # Replace each pixel with its cluster center
    clustered_img = kmeans.cluster_centers_[kmeans.labels_].reshape(img_array.shape)
    
    # Convert back to an image
    clustered_img = np.round(clustered_img).astype(np.uint8)
    return clustered_img, kmeans

In [5]:
# Map each pixel to a number representing the cluster it belongs to
def map_pixels_to_numbers(clustered_img, kmeans):
    # Create a map of colors to cluster labels (numbers)
    labels = kmeans.labels_.reshape(clustered_img.shape[0], clustered_img.shape[1])
    return labels

In [6]:
# Create the "paint by number" image
def create_paint_by_number_image(labels, kmeans, output_path):
    # Create a blank image where each region is filled with its corresponding number
    paint_by_number = np.zeros(labels.shape, dtype=np.uint8)
    
    # Assign each cluster a unique number
    for i in range(kmeans.n_clusters):
        paint_by_number[labels == i] = i + 1  # Add 1 to make the numbering start at 1
    
    # Convert the result to an image and save it
    paint_by_number_img = Image.fromarray(paint_by_number * 25)  # Scale to make numbers visible
    paint_by_number_img.save(output_path)
    print(f"Paint-by-number image saved at {output_path}")

In [7]:
# Generate a legend showing the color for each cluster
def create_color_legend(kmeans, output_path):
    # Create a legend that shows each cluster color with its corresponding number
    legend_img = np.zeros((300, 400, 3), dtype=np.uint8)
    
    for i, color in enumerate(kmeans.cluster_centers_):
        legend_img[i*30:(i+1)*30, :200, :] = color.astype(np.uint8)
        # Add the number
        plt.text(210, i*30 + 15, f"Region {i+1}", fontsize=12, verticalalignment='center')
    
    legend = Image.fromarray(legend_img)
    legend.save(output_path)
    print(f"Color legend saved at {output_path}")

In [15]:
# Main function to create the "paint by number" sheet
def create_paint_by_number_sheet(image_path, output_image_path, output_legend_path, num_colors=10):
    image = load_image(image_path)
    #max_size = 65536 
    #image.thumbnail((max_size, max_size))
    
    # Reduce colors in the image
    clustered_img, kmeans = reduce_colors(image, num_colors)
    
    # Map the image pixels to numbers based on the clustered colors
    labels = map_pixels_to_numbers(clustered_img, kmeans)
    
    # Create the "paint by number" image and save it
    create_paint_by_number_image(labels, kmeans, output_image_path)
    
    # Create and save the color legend
    create_color_legend(kmeans, output_legend_path)

In [ ]:
 # Limit for each dimension
#img.thumbnail((max_size, max_size))

In [16]:
# Example usage
#input_image_path = "input_photo.jpg"  # Path to your input image
input_image_path = "cat_photo.jpeg" 
#input_image_path.thumbnail((max_size, max_size))
#output_image_path = "paint_by_number_image.jpg"  # Path to save the "paint by number" image
output_image_path = "cat_paint_by_number.jpeg"
#output_legend_path = "color_legend_image.jpg"
output_legend_path = "cat_color_legend.jpeg"
# Path to save the color legend

# Generate the "paint by number" sheet
create_paint_by_number_sheet(input_image_path, output_image_path, output_legend_path, num_colors=10)


Paint-by-number image saved at cat_paint_by_number.jpeg
Color legend saved at cat_color_legend.jpeg


ValueError: Image size of 104294x105388 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>

In [ ]:
plt.imshow(new_image)
plt.axis('off')
plt.show()